In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import gzip
import pandas as pd
from scipy import signal
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from collections import Counter
import warnings
import os
import pickle

In [2]:
rules_mapping = {
    0: [255], 1: [127], 2: [16, 191, 247], 3: [17, 63, 119],
    4: [223], 5: [95], 6: [20, 159, 215], 7: [21, 31, 87],
    8: [64, 239, 253], 9: [65, 111, 125], 10: [80, 175, 245], 11: [47, 81, 117],
    12: [68, 207, 221], 13: [69, 79, 93], 14: [84, 143, 213], 15: [85],
    18: [183], 19: [55], 22: [151], 24: [66, 189, 231], 25: [61, 67, 103], 26: [82, 167, 181],
    27: [39, 53, 83], 28: [70, 157, 199], 29: [71], 30: [86, 135, 149],
    32: [251], 33: [123], 34: [48, 187, 243], 35: [49, 59, 115],
    36: [219], 37: [91], 38: [52, 155, 211], 40: [96, 235, 249],
    41: [97, 107, 121], 42: [112, 171, 241], 43: [113], 44: [100, 203, 217],
    45: [75, 89, 101], 46: [116, 139, 209], 50: [179], 54: [147],
    56: [98, 185, 227], 57: [99], 58: [114, 163, 177], 60: [102, 153, 195],
    62: [118, 131, 145], 72: [237], 73: [109], 74: [88, 173, 229],
    76: [205], 78: [92, 141, 197], 90: [165], 94: [133],
    104: [233], 106: [120, 169, 225], 108: [201], 110: [124, 137, 193],
    122: [161], 126: [129], 128: [254], 130: [144, 190, 246],
    132: [222], 134: [148, 158, 214], 136: [192, 238, 252], 138: [174, 208, 224],
    140: [196, 206, 220], 142: [212], 146: [182], 152: [188, 194, 230],
    154: [166, 180, 210], 156: [198], 160: [250], 162: [176, 186, 242],
    164: [218], 168: [224, 234, 248], 170: [240], 172: [202, 216, 228], 184: [226], 200: [236]
}

# reverse: equivalent_rule -> [list of rules]
reverse_mapping = {}

for rule, equiv_rules in rules_mapping.items():
    for equiv_rule in equiv_rules:
        if equiv_rule not in reverse_mapping:
            reverse_mapping[equiv_rule] = []
        reverse_mapping[equiv_rule].append(rule)

# duplicate
duplicates = {}
for equiv_rule, rules_list in reverse_mapping.items():
    if len(rules_list) > 1:
        duplicates[equiv_rule] = rules_list

if duplicates:
    for equiv_rule, rules_list in sorted(duplicates.items()):
        print(f"\nEquivalent Rule {equiv_rule} mapping to more than one Rule:")
        for rule in rules_list:
            print(f"  - Rule {rule}: {rules_mapping[rule]}")
else:
    print("No repeat rules")


Equivalent Rule 224 mapping to more than one Rule:
  - Rule 138: [174, 208, 224]
  - Rule 168: [224, 234, 248]


In [3]:
wolfram_labels = {
    # Class I (8 rules)
    0: 'I', 8: 'I', 32: 'I', 40: 'I', 128: 'I', 136: 'I', 160: 'I', 168: 'I',
    
    # Class II (65 rules)
    1: 'II', 2: 'II', 3: 'II', 4: 'II', 5: 'II', 6: 'II', 7: 'II', 9: 'II',
    10: 'II', 11: 'II', 12: 'II', 13: 'II', 14: 'II', 15: 'II', 19: 'II', 23: 'II',
    24: 'II', 25: 'II', 26: 'II', 27: 'II', 28: 'II', 29: 'II', 33: 'II', 34: 'II',
    35: 'II', 36: 'II', 37: 'II', 38: 'II', 42: 'II', 43: 'II', 44: 'II', 46: 'II',
    50: 'II', 51: 'II', 56: 'II', 57: 'II', 58: 'II', 62: 'II', 72: 'II', 73: 'II',
    74: 'II', 76: 'II', 77: 'II', 78: 'II', 94: 'II', 104: 'II', 108: 'II',
    130: 'II', 132: 'II', 134: 'II', 138: 'II', 140: 'II', 142: 'II', 152: 'II',
    154: 'II', 156: 'II', 162: 'II', 164: 'II', 170: 'II', 172: 'II', 178: 'II',
    184: 'II', 200: 'II', 204: 'II', 232: 'II',
    
    # Class III (11 rules)
    18: 'III', 22: 'III', 30: 'III', 45: 'III', 60: 'III', 90: 'III', 105: 'III',
    122: 'III', 126: 'III', 146: 'III', 150: 'III',
    
    # Class IV (4 rules)
    41: 'IV', 54: 'IV', 106: 'IV', 110: 'IV'
}